# Solution of the olympic task "Спасти парашютиста"

### The description

Парашютист выпрыгнул из самолета, но у него парашют раскрылся неполностью. К счастью рядом оказался спасательный дрон. Для спасения дрону необходимо рассчитать траекторию падающего парашютиста. Координата высоты вычисляема тривиально, однако дрону необходимо вычислить плоскостные координаты $(x, y)$. Есть траектория изменения, по ним необходимо вычислить последующие координаты дрона.
Есть примерная формула изменения координат, она зависит от двух коэффициентов $k_1$ и $k_2$ и случайного шума:

#### $ x = k_2cos(k_1t) + n $,  $ y = k_1sin(k_2t) + n $, где $t$ - № секунды ($1000≤t≤1999$), а $n$ - случайный шум.

На вход подается `list` в $1000$ пар координат ($x, y$) в первую тысячу секунд (с $0$ по $999$-ю).
Пример входных данных:

$ [(0.8990863593574939, 8.475387292276096e-07), (0.8089593833113505, 0.3534820061950688), (0.5566465706027002, 0.4399620826711653), ..., (0.19273411906364982, 0.19411748737324), (-0.20981822525965102, -0.19835273653070376), (-0.5703058016775612, -0.44099688186601116)] $

На выходе программа должна вывести `list` в $1000$ пар координат ($x, y$) во вторую тысячу секунд (с $1000$ по $1999$-ю).
Пример выходных данных:

$ [(0.8990863593574939, 8.475387292276096e-07), (0.8089593833113505, 0.3534820061950688), (0.5566465706027002, 0.4399620826711653), ..., (0.19273411906364982, 0.19411748737324), (-0.20981822525965102, -0.19835273653070376), (-0.5703058016775612, -0.44099688186601116)] $

### Import libraries

In [1]:
import torch
from tqdm import tqdm
%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


### Solution

In [2]:
def dataloader(k1: float, k2: float) -> tuple:
    t = torch.tensor([t for t in range(1000)], dtype=torch.float)
    x = k2 * torch.cos(k1 * t) + torch.randn(1000)
    y = k1 * torch.sin(k2 * t) + torch.randn(1000)
    return t, torch.stack([x, y], dim=0).requires_grad_(True)

In [3]:
def model(k1k2: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    x = k1k2[1] * torch.cos(k1k2[0] * t)
    y = k1k2[0] * torch.sin(k1k2[1] * t)
    return torch.stack([x, y], dim=0).requires_grad_(True)

In [4]:
def clip_grad(grad: torch.Tensor, max_grad_len: float) -> tuple:
    grad_len = torch.norm(grad)
    if grad_len < max_grad_len:
        return grad, grad_len
    return (grad / grad_len) * max_grad_len, grad_len

In [13]:
def main(alpha=.01, step=1, max_grad_len=1., desired_grad_len=1e-3) -> tuple:
    t, y = dataloader(3., 5.)
    k1k2 = torch.tensor([1, 1], dtype=torch.float, requires_grad=True)
    grad_len = desired_grad_len + 1
    while grad_len > desired_grad_len:
        yy = model(k1k2, t)
        mse = ((yy - y)**2).mean()
        mse.backward()
        clipped_grad, grad_len = clip_grad(k1k2.grad, max_grad_len)
        k1k2 = (k1k2 - clipped_grad * alpha * step).clone().detach().requires_grad_(True)
        print(k1k2)
    return k1k2[0].item(), k1k2[1].item()

In [14]:
main()

tensor([0.9992, 1.0100], requires_grad=True)
tensor([0.9952, 1.0008], requires_grad=True)
tensor([0.9854, 1.0027], requires_grad=True)
tensor([0.9845, 0.9927], requires_grad=True)
tensor([0.9789, 1.0010], requires_grad=True)
tensor([0.9886, 0.9989], requires_grad=True)
tensor([0.9922, 1.0083], requires_grad=True)
tensor([1.0015, 1.0046], requires_grad=True)
tensor([1.0053, 1.0139], requires_grad=True)
tensor([1.0150, 1.0160], requires_grad=True)
tensor([1.0152, 1.0060], requires_grad=True)
tensor([1.0150, 1.0160], requires_grad=True)
tensor([1.0153, 1.0060], requires_grad=True)
tensor([1.0150, 1.0160], requires_grad=True)
tensor([1.0154, 1.0060], requires_grad=True)
tensor([1.0148, 1.0160], requires_grad=True)
tensor([1.0157, 1.0060], requires_grad=True)
tensor([1.0146, 1.0159], requires_grad=True)
tensor([1.0160, 1.0060], requires_grad=True)
tensor([1.0143, 1.0159], requires_grad=True)
tensor([1.0159, 1.0060], requires_grad=True)
tensor([1.0144, 1.0159], requires_grad=True)
tensor([1.

KeyboardInterrupt: 